In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import tempfile

import tensorflow as tf
import numpy as np

/scratch/kc3172/py3tf/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def deepnn(x):
    
    with tf.name_scope('reshape'):
#         x_image = tf.reshape(x, [-1, 1, 60, 1])
        x_image = tf.reshape(x, [-1, 1, num_features, 1])


    with tf.name_scope('conv1'):
        W_conv1 = weight_variable([1, 25, 1, 32])
        b_conv1 = bias_variable([32])
        h_conv1 = tf.nn.tanh(conv2d(x_image, W_conv1) + b_conv1)


    with tf.name_scope('pool1'):
        h_pool1 = max_pool_2x2(h_conv1)


    with tf.name_scope('conv2'):
        W_conv2 = weight_variable([1, 25, 32, 64])
        b_conv2 = bias_variable([64])
        h_conv2 = tf.nn.tanh(conv2d(h_pool1, W_conv2) + b_conv2)


    with tf.name_scope('pool2'):
        h_pool2 = max_pool_2x2(h_conv2)


    with tf.name_scope('fc1'):

        W_fc1 = weight_variable([ ceildiv(ceildiv(num_features,stride),stride)*64, 1024])
        b_fc1 = bias_variable([1024])

        h_pool2_flat = tf.reshape(h_pool2, [-1, ceildiv(ceildiv(num_features,stride),stride)*64])
        h_fc1 = tf.nn.tanh(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
#         h_fc1 = tf.matmul(h_pool2_flat, W_fc1) + b_fc1

    with tf.name_scope('dropout'):
        keep_prob = tf.placeholder(tf.float32)
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    with tf.name_scope('fc2'):
        W_fc2 = weight_variable([1024, num_class])
        b_fc2 = bias_variable([num_class])

        y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2 
    return y_conv, keep_prob


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def ceildiv(a, b):
    return -(-a // b)

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 1, stride, 1],
                        strides=[1, 1, stride, 1], padding='SAME')


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


In [3]:
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets('/tmp/tensorflow/mnist/input_data', one_hot=True)

import input_data
totrain = './data/skype-sub-session/session-1'
mnist = input_data.read_data_sets(totrain, one_hot=True)


Extracting ./data/skype-sub-session/session-1/1pkts-subflow-skype-train-images-idx2-ubyte.gz
num_images & leng of image:
3500 73
Extracting ./data/skype-sub-session/session-1/1pkts-subflow-skype-train-labels-idx1-ubyte.gz
3500
Extracting ./data/skype-sub-session/session-1/1pkts-subflow-skype-test-images-idx2-ubyte.gz
num_images & leng of image:
876 73
Extracting ./data/skype-sub-session/session-1/1pkts-subflow-skype-test-labels-idx1-ubyte.gz
876


In [4]:
## kc 7.13 perturb test
import random 

def print_counter(train_y_, test_y_):
    print("test count:  %s" %Counter(np.argmax(test_y_,1)))
    print("train count: %s" %Counter(np.argmax(train_y_,1)))

def sample_test(train_x, train_y):
#     idxrange = range(np.shape(train_x)[0])
#     idx = sorted(random.sample(idxrange,10000))
    np.random.seed(1)
    idx = np.random.choice(np.arange(len(train_x)), 100, replace=False)
    
    ttemp_test = train_x[idx,:]
    test = ttemp_test
    test_label = train_y[idx,:]
    
    train = np.delete(train_x, idx, 0)
    train_label = np.delete(train_y, idx, 0)
    
#     temp = ttemp[:,:-4]
#     temp_label = ttemp[:,-4:]

    return test, test_label, train, train_label
#     return test, test_label

# test_x = mnist.test.images
# test_y = mnist.test.labels
# train_x = mnist.train.images
# train_y = mnist.train.labels

test_x, test_y, train_x, train_y = sample_test(mnist.train.images, mnist.train.labels)
# test_x, test_y = sample_test(mnist.train.images, mnist.train.labels)

In [5]:
# np.shape(mnist.train.images)
np.shape(train_x)

(3400, 60)

In [6]:
# from tensorflow.contrib.learn.python.learn.datasets.mnist import DataSet
# TRAIN_ = DataSet(mnist.train.images[:5000], mnist.train.labels[:5000], reshape=False)
# TEST_ = DataSet(mnist.test.images, mnist.test.labels, reshape=False)

In [7]:
# np.shape(TRAIN_.images)

In [8]:
# import csv
# import random

# def read_csv(dir):
#     # filename = './data_split_train/pkt1.csv'
#     temp = {}
#     temp_test = {}
#     for i in range(1, 11):
#         name = "pkt" + str(i)
#         temp[name] = []
#         filename = dir + '/pkt' + str(i) + '.csv'

#         with open(filename, 'rt') as f:
#             reader = csv.reader(f)
#             try:
#                 ttemp = np.array(list(reader), dtype= np.float32)
#                 # print(np.shape(ttemp))
#             except csv.Error as e:
#                 sys.exit('file %s, line %d: %s' % (filename, reader.line_num, e))

#         #7.12
#         idxrange = range(np.shape(ttemp)[0])
#         idx = sorted(random.sample(idxrange,950))
        
        
#         ttemp_test = ttemp[idx,:]
#         temp_test[name] = ttemp_test[:,:-4]
#         temp_test_label = ttemp_test[:,-4:]
#         temp[name] = ttemp[:,:-4]
#         temp_label = ttemp[:,-4:]

#         # # 7.11
#         # ttemp_test = ttemp[np.random.randint(ttemp.shape[0], size=100), :]
#         #
#         # temp[name] = ttemp[:, :-4]
#         # # print(np.shape(temp[name]))
#         # temp_label = ttemp[:, -4:]
#         #
#         # #7.11
#         # # temp_test[name] = temp[name][400:500,:]
#         # # print(np.shape(ttemp_test))
#         # temp_test[name] = ttemp_test[:,:-4]
#         #
#         # temp_test_label = ttemp_test[:,-4:]
#         # print(np.shape(temp_test[name]),np.shape(temp_test_label))

#     return temp, temp_label, temp_test, temp_test_label

# train_dir = "./data_split_train_712"
# test_dir = "./data_split_test_712"
# print("-----------------------------------")
# print("reading csv")
# train_x, train_y, test_x, test_y = read_csv(train_dir)

In [9]:
from sklearn.utils import shuffle

def shuffle_dataset(train_x, train_y):
    x, y = shuffle(train_x, train_y)
    
    return x, y

In [10]:
# Import data


num_features = 60
num_class = 4
epoch = 3000
batch_size = 512
stride = 1

# Create the model
x = tf.placeholder(tf.float32, [None, num_features])

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, num_class])



# The raw formulation of cross-entropy,
#
#   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
#                                 reduction_indices=[1]))
#
# can be numerically unstable.
#
# So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
# outputs of 'y', and then average across the batch.
y_conv, keep_prob = deepnn(x)

with tf.name_scope('loss'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_,
                                                            logits=y_conv)
cross_entropy = tf.reduce_mean(cross_entropy)

with tf.name_scope('adam_optimizer'):
    train_step = tf.train.AdamOptimizer(1e-4, beta1=0.9, beta2=0.9999).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
    accuracy = tf.reduce_mean(correct_prediction)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
  # Train
    
tr_acc =[]
tr_loss =[]
ts_acc =[]
ts_loss =[]

# change batch method
# for i in range(20001):
# #     batch = TRAIN_.next_batch(50)
#     batch = mnist.train.next_batch(50)
# #     batch_x = train_x[i*batch_size:(i+1)*batch_size]
#     sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
#     #train
#     if i%100 == 0:
# #         print(i%100)
        
#         tr_acc.append(accuracy.eval(feed_dict={x: mnist.train.images, y_: mnist.train.labels, keep_prob:1.0}))
#         tr_loss.append(cross_entropy.eval(feed_dict={x: mnist.train.images, y_: mnist.train.labels, keep_prob:1.0}))
#         #test
#         ts_acc.append(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob:1.0}))
#         ts_loss.append(cross_entropy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob:1.0}))
        
#         s = "step %d, train accuracy %s, train loss %s, test accuracy %s, test loss %s" % (i, 
#          tr_acc[-1], 
#          tr_loss[-1],
#          ts_acc[-1], 
#          ts_loss[-1])
#         print(s)

for j in range(epoch + 1):
    tot_batch = int(train_x.shape[0] / batch_size)
#     print(tot_batch)
    for i in range (tot_batch):
        batch_x = train_x[i*batch_size:(i+1)*batch_size]
        batch_y = train_y[i*batch_size:(i+1)*batch_size]
        ## 7.11
        train_step.run(feed_dict={x: batch_x, y_: batch_y, keep_prob: 1.0})       

        if i == tot_batch-1 and j%5==0:
            print("EPOCH: %d" % j)
            ## kc, change batch to whole
            tr_acc.append(accuracy.eval(feed_dict={x: train_x, y_: train_y, keep_prob:1.0}))
            tr_loss.append(cross_entropy.eval(feed_dict={x: train_x, y_: train_y, keep_prob:1.0}))
            #test
            ts_acc.append(accuracy.eval(feed_dict={x: test_x, y_: test_y, keep_prob:1.0}))
            ts_loss.append(cross_entropy.eval(feed_dict={x: test_x, y_: test_y, keep_prob:1.0}))

            s = "step %d, train accuracy %s, train loss %s, test accuracy %s, test loss %s" % (i, 
             tr_acc[-1], 
             tr_loss[-1],
             ts_acc[-1], 
             ts_loss[-1])
            print(s)
    train_step.run(feed_dict={x: train_x[tot_batch*batch_size:], y_: train_y[tot_batch*batch_size:], keep_prob: 1.0})
    train_x, train_y = shuffle_dataset(train_x, train_y)
    
  # Test trained model

print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                      y_: mnist.test.labels, keep_prob:1.0}))

# if __name__ == '__main__':
#   parser = argparse.ArgumentParser()
#   parser.add_argument('--data_dir', type=str, default='/tmp/tensorflow/mnist/input_data',
#                       help='Directory for storing input data')
#   FLAGS, unparsed = parser.parse_known_args()
#   tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

EPOCH: 0
step 5, train accuracy 0.50941175, train loss 1.1687529, test accuracy 0.5, test loss 1.0782814
EPOCH: 5
step 5, train accuracy 0.64941174, train loss 0.7965515, test accuracy 0.65, test loss 0.7707038
EPOCH: 10
step 5, train accuracy 0.6814706, train loss 0.74392843, test accuracy 0.68, test loss 0.7602853
EPOCH: 15
step 5, train accuracy 0.69, train loss 0.72086203, test accuracy 0.69, test loss 0.7534536
EPOCH: 20
step 5, train accuracy 0.6861765, train loss 0.695042, test accuracy 0.66, test loss 0.74761695
EPOCH: 25
step 5, train accuracy 0.72, train loss 0.6567607, test accuracy 0.68, test loss 0.72594404
EPOCH: 30
step 5, train accuracy 0.71882355, train loss 0.63711977, test accuracy 0.67, test loss 0.7260788
EPOCH: 35
step 5, train accuracy 0.72794116, train loss 0.63010037, test

EPOCH: 365
step 5, train accuracy 0.9270588, train loss 0.23457655, test accuracy 0.87, test loss 0.4966375
EPOCH: 370
step 5, train accuracy 0.93558824, train loss 0.22945751, test accuracy 0.88, test loss 0.49684036
EPOCH: 375
step 5, train accuracy 0.9264706, train loss 0.23207285, test accuracy 0.86, test loss 0.5137228
EPOCH: 380
step 5, train accuracy 0.9279412, train loss 0.22524278, test accuracy 0.9, test loss 0.48507622
EPOCH: 385
step 5, train accuracy 0.9352941, train loss 0.22049284, test accuracy 0.87, test loss 0.5030025
EPOCH: 390
step 5, train accuracy 0.93588233, train loss 0.21890607, test accuracy 0.88, test loss 0.51325136
EPOCH: 395
step 5, train accuracy 0.93911767, train loss 0.21293433, test accuracy 0.85, test loss 0.49150708
EPOCH: 400
step 5, train accuracy 0.93647057, train loss 0.2115057, test accuracy 0.88, test loss 0.48761073
EPOCH: 405
step 5, train accuracy 0.93647057, train loss 0.21577127, test accuracy 0.85, test loss 0.52630323
EPOCH: 410
step 5, 

EPOCH: 745
step 5, train accuracy 0.99352944, train loss 0.07213507, test accuracy 0.88, test loss 0.5281766
EPOCH: 750
step 5, train accuracy 0.9932353, train loss 0.07026687, test accuracy 0.87, test loss 0.5309186
EPOCH: 755
step 5, train accuracy 0.99352944, train loss 0.068707034, test accuracy 0.88, test loss 0.5269077
EPOCH: 760
step 5, train accuracy 0.9920588, train loss 0.06862483, test accuracy 0.88, test loss 0.53726524
EPOCH: 765
step 5, train accuracy 0.9941176, train loss 0.067820795, test accuracy 0.89, test loss 0.5239355
EPOCH: 770
step 5, train accuracy 0.995, train loss 0.066125505, test accuracy 0.88, test loss 0.5322874
EPOCH: 775
step 5, train accuracy 0.9941176, train loss 0.0648504, test accuracy 0.89, test loss 0.53533727
EPOCH: 780
step 5, train accuracy 0.99441177, train loss 0.06408471, test accuracy 0.88, test loss 0.53315467
EPOCH: 785
step 5, train accuracy 0.9952941, train loss 0.06304791, test accuracy 0.87, test loss 0.5288285
EPOCH: 790
step 5, train

EPOCH: 1120
step 5, train accuracy 0.99941176, train loss 0.025787188, test accuracy 0.89, test loss 0.56584257
EPOCH: 1125
step 5, train accuracy 0.9997059, train loss 0.025553536, test accuracy 0.88, test loss 0.56684434
EPOCH: 1130
step 5, train accuracy 1.0, train loss 0.02517722, test accuracy 0.89, test loss 0.56656045
EPOCH: 1135
step 5, train accuracy 0.9997059, train loss 0.02490383, test accuracy 0.89, test loss 0.56490815
EPOCH: 1140
step 5, train accuracy 0.9997059, train loss 0.024607612, test accuracy 0.89, test loss 0.5666677
EPOCH: 1145
step 5, train accuracy 0.9997059, train loss 0.024378588, test accuracy 0.89, test loss 0.5687356
EPOCH: 1150
step 5, train accuracy 1.0, train loss 0.024073623, test accuracy 0.89, test loss 0.5669923
EPOCH: 1155
step 5, train accuracy 0.9997059, train loss 0.023783764, test accuracy 0.89, test loss 0.56706476
EPOCH: 1160
step 5, train accuracy 0.9997059, train loss 0.023538848, test accuracy 0.89, test loss 0.56941724
EPOCH: 1165
step 

EPOCH: 1510
step 5, train accuracy 1.0, train loss 0.011951706, test accuracy 0.89, test loss 0.5885828
EPOCH: 1515
step 5, train accuracy 1.0, train loss 0.011818008, test accuracy 0.89, test loss 0.5851324
EPOCH: 1520
step 5, train accuracy 1.0, train loss 0.011757834, test accuracy 0.88, test loss 0.5869515
EPOCH: 1525
step 5, train accuracy 1.0, train loss 0.011627766, test accuracy 0.89, test loss 0.5865792
EPOCH: 1530
step 5, train accuracy 1.0, train loss 0.011533192, test accuracy 0.89, test loss 0.5887193
EPOCH: 1535
step 5, train accuracy 1.0, train loss 0.011492839, test accuracy 0.89, test loss 0.58538806
EPOCH: 1540
step 5, train accuracy 1.0, train loss 0.011348852, test accuracy 0.89, test loss 0.5866542
EPOCH: 1545
step 5, train accuracy 1.0, train loss 0.011242918, test accuracy 0.89, test loss 0.5896008
EPOCH: 1550
step 5, train accuracy 1.0, train loss 0.01117122, test accuracy 0.88, test loss 0.59026355
EPOCH: 1555
step 5, train accuracy 1.0, train loss 0.0110900495

EPOCH: 1905
step 5, train accuracy 1.0, train loss 0.0066007944, test accuracy 0.88, test loss 0.59970766
EPOCH: 1910
step 5, train accuracy 1.0, train loss 0.006549348, test accuracy 0.89, test loss 0.59864223
EPOCH: 1915
step 5, train accuracy 1.0, train loss 0.006518055, test accuracy 0.89, test loss 0.5985246
EPOCH: 1920
step 5, train accuracy 1.0, train loss 0.0064676376, test accuracy 0.89, test loss 0.59903556
EPOCH: 1925
step 5, train accuracy 1.0, train loss 0.0064391987, test accuracy 0.89, test loss 0.5990691
EPOCH: 1930
step 5, train accuracy 1.0, train loss 0.0063850633, test accuracy 0.89, test loss 0.60036224
EPOCH: 1935
step 5, train accuracy 1.0, train loss 0.0063474644, test accuracy 0.89, test loss 0.59987664
EPOCH: 1940
step 5, train accuracy 1.0, train loss 0.0063098706, test accuracy 0.89, test loss 0.5995149
EPOCH: 1945
step 5, train accuracy 1.0, train loss 0.006257241, test accuracy 0.89, test loss 0.6001989
EPOCH: 1950
step 5, train accuracy 1.0, train loss 0.

EPOCH: 2300
step 5, train accuracy 1.0, train loss 0.004117438, test accuracy 0.88, test loss 0.6119911
EPOCH: 2305
step 5, train accuracy 1.0, train loss 0.0040939986, test accuracy 0.88, test loss 0.611582
EPOCH: 2310
step 5, train accuracy 1.0, train loss 0.0040694256, test accuracy 0.88, test loss 0.61203057
EPOCH: 2315
step 5, train accuracy 1.0, train loss 0.0040539233, test accuracy 0.88, test loss 0.6134024
EPOCH: 2320
step 5, train accuracy 1.0, train loss 0.0040258383, test accuracy 0.88, test loss 0.6109305
EPOCH: 2325
step 5, train accuracy 1.0, train loss 0.0040059416, test accuracy 0.88, test loss 0.61215883
EPOCH: 2330
step 5, train accuracy 1.0, train loss 0.0039819116, test accuracy 0.88, test loss 0.61175394
EPOCH: 2335
step 5, train accuracy 1.0, train loss 0.003978216, test accuracy 0.89, test loss 0.6116661
EPOCH: 2340
step 5, train accuracy 1.0, train loss 0.003940806, test accuracy 0.88, test loss 0.6123158
EPOCH: 2345
step 5, train accuracy 1.0, train loss 0.003

EPOCH: 2690
step 5, train accuracy 1.0, train loss 0.0027844987, test accuracy 0.88, test loss 0.6211565
EPOCH: 2695
step 5, train accuracy 1.0, train loss 0.0027719452, test accuracy 0.88, test loss 0.61927795
EPOCH: 2700
step 5, train accuracy 1.0, train loss 0.0027609793, test accuracy 0.88, test loss 0.62082464
EPOCH: 2705
step 5, train accuracy 1.0, train loss 0.0027466763, test accuracy 0.88, test loss 0.6207352
EPOCH: 2710
step 5, train accuracy 1.0, train loss 0.0027361147, test accuracy 0.88, test loss 0.6206764
EPOCH: 2715
step 5, train accuracy 1.0, train loss 0.0027213243, test accuracy 0.88, test loss 0.62112373
EPOCH: 2720
step 5, train accuracy 1.0, train loss 0.0027093454, test accuracy 0.88, test loss 0.6200907
EPOCH: 2725
step 5, train accuracy 1.0, train loss 0.002696886, test accuracy 0.88, test loss 0.62126255
EPOCH: 2730
step 5, train accuracy 1.0, train loss 0.0026852698, test accuracy 0.88, test loss 0.6225954
EPOCH: 2735
step 5, train accuracy 1.0, train loss 0

In [11]:
sess.close()

In [12]:
import matplotlib.pyplot as plt
import numpy
plt.subplot(211)
g = plt.figure(1)
## change x range
length_acc = numpy.shape(tr_loss)[0]
plt.plot(list(range(length_acc)), list(tr_loss), color='r', label='training loss')
plt.plot(list(range(length_acc)), list(ts_loss), color='g', label='testing loss')

plt.legend()
# plt.title('SVM accuracy curve')
# plt.xlabel('# of features chosen')
plt.ylabel('loss')
plt.grid()
plt.subplot(212)
length_acc = numpy.shape(tr_loss)[0]
plt.plot(list(range(length_acc)), list(tr_acc), color='r', label='training acc')
plt.plot(list(range(length_acc)), list(ts_acc), color='g', label='testing acc')
plt.legend()
plt.ylim(ymax=1)  # adjust the max leaving min unchanged
# plt.title('SVM accuracy curve')
# plt.xlabel('# of features chosen')
plt.ylabel('acc')
plt.grid()
plt.show()

<Figure size 640x480 with 2 Axes>

In [13]:
ts_acc[-1]

0.88